In [9]:
!pip install pandas numpy xgboost flask flask-cors gunicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.0 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv("Fish_data.csv")  # Ensure file is uploaded

# Encode categorical columns
if "Species" in df.columns:
    df["Species"] = LabelEncoder().fit_transform(df["Species"])

# Split data into features (X) and target (y)
X = df.drop(columns=["Weight"])  # Features
y = df["Weight"]  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "model.pkl")
print("✅ Model saved as 'model.pkl'")


✅ Model saved as 'model.pkl'


In [12]:
from flask import Flask, request, jsonify, render_template
import joblib
import numpy as np
from flask_cors import CORS

# Load trained model
model = joblib.load("model.pkl")

# Initialize Flask app
app = Flask(__name__)
CORS(app)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    features = np.array(data["features"]).reshape(1, -1)  # Convert to 2D array
    prediction = model.predict(features)[0]
    return jsonify({"predicted_weight": round(prediction, 2)})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
